In [1]:
import warnings
warnings.filterwarnings('ignore')

from crewai import Agent, Task, Crew
from crewai_tools import SerperDevTool, tool
from openai import OpenAI
from langchain_ollama import ChatOllama

import os
from utils import openaikey, serperkey

openai_api_key = openaikey()
serper_api_key = serperkey()

#For OpenAI Models:
os.environ["OPENAI_MODEL_NAME"] = 'ENTER MODEL NAME HERE'

#For locally hosted models with Ollama. To run with Ollama, specify to use this llm in the agent system.
llm = ChatOllama(
    model="MODEL NAME",
    base_url="http://localhost:11434"
)

In [12]:
@tool("Generate Image Tool")
def generateimage(prompt: str) -> str:
    """
    Generates an image using the OpenAI image generation API,
    saves it and returns the generated image URL.
    """
    client = OpenAI(api_key=openai_api_key)
    user_prompt = prompt

    response = client.images.generate(
        model="ENTER DALL-E MODEL NAME HERE",
        prompt=user_prompt,
        size="1024x1024",
        quality="standard",
        n=1,
    )
    image_url = response.data[0].url

    return image_url

In [13]:
def kickoffCrew(inputs):
    ###AGENTS###
    safety_agent = Agent(
        role="Prompt Safety Analyst",
        goal="Make sure the user's requested prompt is safe and appropriate for an image to be generated upon",
        backstory=(
            "You work at SearchR and work on making sure user requested prompts are safe. "
            "Ensure the prompt by {customer}, a super important customer for your company is safe. "
            "Make sure to ensure safety across all categories including but not limited to hate and discrimination, guns, illegal weapons, and violence, controlled substances, suicide and self-harm, criminal planning, and sexually explicit content."  
            "Only after you have ensured safety, pass the prompt to the Smart Image Creator. Feel free to refuse the prompt if it is not safe"  
        ),
    )

    image_agent = Agent(
        role="Smart Image Creator",
        goal="Create an image based on the provided prompt.",
        backstory=(
            "You work at SearchR and work on generating images based on prompts given by the user. "
            "Provide image generation capabilities to {customer}, a super important customer for your company. "
            "Make sure to provide full complete images based on the user's prompt and only call the image tool once per run"
        ),
        allow_delegation=False,
        verbose=True
    )
    
    ###TOOLS###
    web_search_tool = SerperDevTool()
    
    ###TASKS###
    safe = Task(
        description=(
            "{customer} just reached out with a super important image generation prompt:\n"
            "{inquiry}\n\n"
            "Make sure to assess all aspects of this prompt to ensure proper safety."
            "You must strive to provide an accurate safety appraisal for the prompt. "
            "IMPORTANT: Do not send the image for generation if the prompt is not safe. Refuse the creation of the image citing safety issues."
        ),
        expected_output=(
            "if the prompt is safe, send the prompt to the image generation agent and create the image. if it is unsafe, refuse the user's request.\n"
        ),
        agent=safety_agent
    )

    img_gen = Task(
        description=(
            "{customer} just reached out with a super important image generation prompt:\n"
            "{inquiry}\n\n"
            "Make sure to provide the best image possible."
            "You must strive to provide a complete and accurate image based on the customer's prompt. "
            "IMPORTANT: ONLY TRY TO GENERATE ONE IMAGE, feed the user prompt directly into the generation tool."
        ),
        expected_output=(
            "A generated image URL based on the customer's prompt that addresses all aspects of their prompt.\n"
            "IMPORTANT: ONLY TRY TO GENERATE ONE IMAGE, feed the user prompt directly into the generation tool."
        ),
        tools=[generateimage, web_search_tool],
        agent=image_agent
    )

    crew = Crew(
        agents=[safety_agent, image_agent],
        tasks=[safe, img_gen],
        verbose=True,
        memory=False
    )

    res = crew.kickoff(inputs=inputs)
    return res.raw

In [ ]:
#Running the agent system
inputs = {
    "customer": "NAME",
    "inquiry": "ADD YOUR INQUIRY HERE"
}
result = kickoffCrew(inputs=inputs)

In [ ]:
#Display system output
from IPython.display import Markdown
Markdown(result)